<a href="https://colab.research.google.com/github/smze/Virtual_Fence1_Final/blob/main/Virtual_Fence_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

نصب و وارد کردن تمام کتابخانه‌های مورد نیاز برای پردازش ویدیو، YOLOv8، VLM، MobileNet و مدیریت دیتاست

In [ ]:
###Part 1: نصب و وارد کردن کتابخانه‌ها
########################################
# نصب کتابخانه‌ها
!pip install -q ultralytics opencv-python-headless tqdm pillow pyyaml torch torchvision torchaudio matplotlib seaborn albumentations scikit-learn

# وارد کردن کتابخانه‌ها
import os, json, random, shutil, zipfile, yaml
from glob import glob
from tqdm import tqdm
from PIL import Image
import cv2
import torch
from google.colab import drive, files


Part 2: مونت کردن Google Drive و استخراج دیتاست سفارشی

دیتاست سفارشی از Drive استخراج می‌شود و آماده ادغام با CrowdHuman است


In [ ]:
# مونت کردن Google Drive
drive.mount('/content/drive', force_remount=True)

# مسیر فایل ZIP دیتاست سفارشی
ZIP_PATH = "/content/drive/MyDrive/Virtual_Fence/my_custom_yolo.zip"
BASE_CUSTOM = "/content/my_custom_yolo"
os.makedirs(BASE_CUSTOM, exist_ok=True)

# استخراج دیتاست سفارشی
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(BASE_CUSTOM)
print("✅ Custom dataset extracted from Drive!")


دیتاست CrowdHuman COCO دانلود و مسیرهای train/val آماده می‌شود

پوشه‌های YOLO برای ذخیره تصاویر و برچسب‌ها ساخته می‌شود

In [ ]:
##Part 3: دانلود و آماده‌سازی CrowdHuman COCO
# آپلود kaggle.json برای دسترسی به API Kaggle
files.upload()  # فایل kaggle.json را انتخاب کنید

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# دانلود CrowdHuman COCO
!kaggle datasets download -d kakkimch/crowdhuman-coco -p /content
!mkdir -p /content/crowdhuman
!unzip -q /content/crowdhuman-coco.zip -d /content/crowdhuman

# مسیر فایل‌ها
CROWD_PATH = "/content/crowdhuman/CrowdHuman"
TRAIN_ODGT = os.path.join(CROWD_PATH, "annotation_train.odgt")
VAL_ODGT = os.path.join(CROWD_PATH, "annotation_val.odgt")
IMAGES_DIR_TRAIN = os.path.join(CROWD_PATH, "Images/Train")
IMAGES_DIR_VAL = os.path.join(CROWD_PATH, "Images/Val")

# مسیر خروجی نهایی YOLO
OUTPUT_BASE = "/content/CrowdHuman_YOLO"
LABELS_TRAIN = os.path.join(OUTPUT_BASE, "labels/train")
LABELS_VAL = os.path.join(OUTPUT_BASE, "labels/val")
IMAGES_TRAIN = os.path.join(OUTPUT_BASE, "images/train")
IMAGES_VAL = os.path.join(OUTPUT_BASE, "images/val")

# ساخت پوشه‌ها
for p in [LABELS_TRAIN, LABELS_VAL, IMAGES_TRAIN, IMAGES_VAL]:
   os.makedirs(p, exist_ok=True)


In [ ]:
###Part 4: تابع تبدیل .odgt به YOLO و کپی تصاویر
def convert_odgt_to_yolo(odgt_file, output_dir, images_dir, images_out_dir):
    with open(odgt_file, "r") as f:
        lines = f.readlines()

    print(f"📄 Found {len(lines)} annotation lines in {odgt_file}")

    for line in tqdm(lines, desc=f"Processing {os.path.basename(odgt_file)}"):
        data = json.loads(line)
        image_name = data["ID"] + ".jpg"
        img_path = os.path.join(images_dir, image_name)

        if not os.path.exists(img_path):
            continue

        # خواندن اندازه تصویر
        try:
            with Image.open(img_path) as img:
                w, h = img.size
        except:
            continue

        # کپی تصویر به پوشه خروجی
        shutil.copy(img_path, os.path.join(images_out_dir, image_name))

        # ساخت فایل txt YOLO
        label_path = os.path.join(output_dir, image_name.replace(".jpg", ".txt"))
        with open(label_path, "w") as out:
            for gtbox in data["gtboxes"]:
                if "head_attr" in gtbox and gtbox["head_attr"].get("ignore", 0) == 1:
                    continue
                if "fbox" not in gtbox:
                    continue
                x, y, bw, bh = gtbox["fbox"]
                x_center = (x + bw / 2) / w
                y_center = (y + bh / 2) / h
                w_norm = bw / w
                h_norm = bh / h
                class_id = 0
                out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")


In [ ]:
##Part 5: اجرای تبدیل برای CrowdHuman
convert_odgt_to_yolo(TRAIN_ODGT, LABELS_TRAIN, IMAGES_DIR_TRAIN, IMAGES_TRAIN)
convert_odgt_to_yolo(VAL_ODGT, LABELS_VAL, IMAGES_DIR_VAL, IMAGES_VAL)
print("✅ CrowdHuman converted to YOLO format successfully!")


In [ ]:
##Part 6: آماده‌سازی دیتاست سفارشی و ادغام
SRC_IMAGES = os.path.join(BASE_CUSTOM, "images")
SRC_LABELS = os.path.join(BASE_CUSTOM, "labels")

all_images = glob(SRC_IMAGES + "/*.jpg")
random.shuffle(all_images)
split_idx = int(0.8 * len(all_images))  # 80% train, 20% val

for i, img_path in enumerate(all_images):
    fname = os.path.basename(img_path)
    lbl_path = os.path.join(SRC_LABELS, fname.replace(".jpg", ".txt"))
    if i < split_idx:
        shutil.move(img_path, IMAGES_TRAIN + "/" + fname)
        shutil.move(lbl_path, LABELS_TRAIN + "/" + fname.replace(".jpg", ".txt"))
    else:
        shutil.move(img_path, IMAGES_VAL + "/" + fname)
        shutil.move(lbl_path, LABELS_VAL + "/" + fname.replace(".jpg", ".txt"))

print("✅ Custom dataset merged into CrowdHuman successfully!")


Part 7: ساخت train.txt، val.txt و data.yaml

In [ ]:
##Part 7: ساخت train.txt، val.txt و data.yaml
TRAIN_TXT = os.path.join(OUTPUT_BASE, "train.txt")
VAL_TXT = os.path.join(OUTPUT_BASE, "val.txt")

with open(TRAIN_TXT, "w") as f:
    for img in glob(IMAGES_TRAIN + "/*.jpg"):
        f.write(img + "\n")

with open(VAL_TXT, "w") as f:
    for img in glob(IMAGES_VAL + "/*.jpg"):
        f.write(img + "\n")

DATA_YAML = os.path.join(OUTPUT_BASE, "crowdhuman_merged.yaml")
data_dict = {
    'train': TRAIN_TXT,
    'val': VAL_TXT,
    'nc': 1,
    'names': ['person']
}
with open(DATA_YAML, 'w') as f:
    yaml.dump(data_dict, f)

print("✅ Dataset prepared and YAML created at:", DATA_YAML)


In [ ]:
##Part 8: آموزش YOLOv8 با وزن‌های pre-trained
##وزن‌های از پیش آموزش دیده (yolov8n.pt) استفاده می‌شوند
###آموزش روی دیتاست ترکیبی CrowdHuman + دیتاست سفارشی###

# نصب YOLOv8
!pip install -U ultralytics

# آموزش YOLOv8n با دیتاست ترکیبی
!yolo train \
data=/content/CrowdHuman_YOLO/crowdhuman_merged.yaml \
model=yolov8n.pt \
epochs=50 \
imgsz=640 \
batch=16 \
name=crowdhuman_merged


In [ ]:
##Part 9: آماده‌سازی منطقه مشخص شده (Virtual Fence
# تعریف منطقه مستطیلی در ویدیو
fence_coords = (100, 200, 500, 600)  # (x1, y1, x2, y2) قابل تغییر

def draw_fence(frame, coords=fence_coords, color=(0,255,0), thickness=2):
    cv2.rectangle(frame, (coords[0], coords[1]), (coords[2], coords[3]), color, thickness)


In [ ]:
##Part 10: تعریف کلاس شمارنده و ردیاب برای افراد
class PersonCounter:
    def __init__(self, fence_coords):
        self.fence_coords = fence_coords
        self.counted_ids = set()
        self.total_count = 0

    def update(self, detections):
        # detections: list of dicts with 'id', 'bbox' keys
        for det in detections:
            cx = (det['bbox'][0] + det['bbox'][2]) // 2
            cy = (det['bbox'][1] + det['bbox'][3]) // 2
            if (self.fence_coords[0] <= cx <= self.fence_coords[2] and
                self.fence_coords[1] <= cy <= self.fence_coords[3] and
                det['id'] not in self.counted_ids):
                self.counted_ids.add(det['id'])
                self.total_count += 1
        return self.total_count



In [ ]:
###Part 11: پیاده‌سازی سه رویکرد

In [ ]:
##Cell A — توابع مشترک و کلاس Tracker/Counter
(این سلول اول اجرا شود — توابع و کلاس‌ها بین همه روش‌ها مشترک‌اند)
# ===========================
# Cell A: Shared utilities
# ===========================
import time, math
import numpy as np
import cv2
from collections import OrderedDict

def iou(boxA, boxB):
    # box: [x1,y1,x2,y2]
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA); interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = max(0, (boxA[2]-boxA[0])) * max(0, (boxA[3]-boxA[1]))
    boxBArea = max(0, (boxB[2]-boxB[0])) * max(0, (boxB[3]-boxB[1]))
    denom = float(boxAArea + boxBArea - interArea + 1e-6)
    return interArea / denom if denom>0 else 0.0

class SimpleTracker:
    """
    Simple IoU-based tracker:
    - Keeps dictionary of tracked objects: id -> {'bbox':bbox, 'age':0, 'missed':0, 'counted':False}
    - For each frame: match detections to existing objects by IoU threshold
    - Unmatched detections -> new IDs
    - Unmatched tracks -> increase missed count, remove if missed > max_missed
    """
    def __init__(self, iou_threshold=0.3, max_missed=5):
        self.next_id = 0
        self.tracks = OrderedDict()
        self.iou_threshold = iou_threshold
        self.max_missed = max_missed

    def update(self, detections):
        # detections: list of bboxes [[x1,y1,x2,y2], ...] (integers)
        updated = OrderedDict()
        used_det = set()

        # Match existing tracks -> detections by best IoU
        for tid, t in list(self.tracks.items()):
            best_iou = 0.0; best_j = -1
            for j, det in enumerate(detections):
                if j in used_det: continue
                val = iou(t['bbox'], det)
                if val > best_iou:
                    best_iou = val; best_j = j
            if best_iou >= self.iou_threshold and best_j >= 0:
                # matched
                updated[tid] = {'bbox': detections[best_j], 'age': t['age']+1, 'missed': 0, 'counted': t.get('counted', False)}
                used_det.add(best_j)
            else:
                # no match -> increase missed
                if t['missed']+1 <= self.max_missed:
                    updated[tid] = {'bbox': t['bbox'], 'age': t['age']+1, 'missed': t['missed']+1, 'counted': t.get('counted', False)}
                # else: drop track

        # Add new detections as new tracks
        for j, det in enumerate(detections):
            if j in used_det: continue
            updated[self.next_id] = {'bbox': det, 'age': 1, 'missed': 0, 'counted': False}
            self.next_id += 1

        # Replace tracks
        self.tracks = updated
        return self.tracks

    def reset(self):
        self.next_id = 0
        self.tracks = OrderedDict()


این tracker ساده و کارا است — سریع، بدون وابستگی به کتابخانه‌های خارجی، و برای کاربردهای real-time مناسب است. از IoU برای اتصال اشیاء در فریم‌ها استفاده می‌کند.

In [ ]:
###Cell B — اجرای YOLOv8n (pre-trained یا fine-tuned) با tracker و شمارش
# ===========================
# Cell B: YOLOv8n inference + SimpleTracker + counting
# ===========================
from ultralytics import YOLO
import os, time

# تنظیم‌ها (ویرایش کن اگر لازم است)
INPUT_VIDEO = "/content/input.mp4"                # مسیر ویدیوی ورودی
OUTPUT_VIDEO_YOLO = "/content/output_yolo.mp4"   # مسیر فایل خروجی
FENCE = (200, 100, 600, 400)                     # (x1,y1,x2,y2) — تنظیم دلخواه
YOLO_WEIGHT = "yolov8n.pt"                       # یا مسیر به weight آموزشی شما

# بارگذاری مدل YOLO
yolo_model = YOLO(YOLO_WEIGHT)

# آماده‌سازی ویدیو
cap = cv2.VideoCapture(INPUT_VIDEO)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS) or 25.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_YOLO, fourcc, fps, (width, height))

# tracker و counter
tracker = SimpleTracker(iou_threshold=0.3, max_missed=5)
counted_ids = set()
total_count = 0

start_time = time.time()
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # run YOLO inference (use small batch by single frame)
    results = yolo_model.predict(frame, conf=0.4, classes=[0], verbose=False)  # only person class
    dets = []
    for res in results:
        # each res.boxes contain many boxes
        boxes = res.boxes.xyxy.cpu().numpy()
        confs = res.boxes.conf.cpu().numpy()
        for b, c in zip(boxes, confs):
            if c < 0.25: continue
            x1,y1,x2,y2 = b.astype(int)
            dets.append([x1,y1,x2,y2])

    # update tracker
    tracks = tracker.update(dets)

    # draw & count
    x1f,y1f,x2f,y2f = FENCE
    cv2.rectangle(frame, (x1f,y1f), (x2f,y2f), (0,255,0), 2)
    for tid, t in tracks.items():
        bx = t['bbox']
        bx1,by1,bx2,by2 = map(int, bx)
        cx = int((bx1+bx2)/2); cy = int((by1+by2)/2)
        # draw
        cv2.rectangle(frame, (bx1,by1),(bx2,by2),(0,0,255),2)
        cv2.circle(frame, (cx,cy), 3, (255,0,0), -1)
        cv2.putText(frame, f"ID:{tid}", (bx1, max(0,by1-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)

        # count if entering fence and not counted
        if (x1f <= cx <= x2f and y1f <= cy <= y2f) and (not t['counted']):
            total_count += 1
            t['counted'] = True
            tracker.tracks[tid]['counted'] = True  # persist

    # overlay counter
    cv2.putText(frame, f"YOLO Count: {total_count}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)
    out.write(frame)

cap.release()
out.release()
elapsed_yolo = time.time() - start_time
print(f"✅ YOLO done — Count: {total_count}, Time: {elapsed_yolo:.2f}s, Output: {OUTPUT_VIDEO_YOLO}")
# store results for benchmark
yolo_results = {'count': total_count, 'time': elapsed_yolo, 'output': OUTPUT_VIDEO_YOLO}


این جریان از وزن YOLO_WEIGHT استفاده می‌کند (می‌توانیم مسیر وزن fine-tuned
  خود را جایگزین yolov8n.pt کنیم).

tracker درون حلقه بروز می‌شود و تمام منطق شمارش (مرکز جعبه داخل fence) در همین سلول انجام می‌شود.

خروجی ویدیو و زمان پردازش ذخیره می‌شود.

In [ ]:
###Cell C — اجرای MobileNet-SSD (ssdlite320_mobilenet_v3_large) + tracker + counting

##(این سلول سبک و سریع است؛ مناسب برای دستگاه‌های محدود — از torchvision pretrained استفاده می‌کند)
# ===========================
# Cell C: MobileNet-SSD (ssdlite320_mobilenet_v3_large) inference + tracking + counting
# ===========================
import torch
from torchvision import transforms
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

# تنظیمات
INPUT_VIDEO = "/content/input.mp4"
OUTPUT_VIDEO_MOB = "/content/output_mobilenet.mp4"
FENCE = (200, 100, 600, 400)   # همان منطقه یا دلخواه

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# بارگذاری مدل از torchvision (pretrained COCO)
mob_model = ssdlite320_mobilenet_v3_large(pretrained=True).to(device)
mob_model.eval()

transform = transforms.Compose([
    transforms.ToTensor()
])

cap = cv2.VideoCapture(INPUT_VIDEO)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS) or 25.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_MOB, fourcc, fps, (width, height))

tracker = SimpleTracker(iou_threshold=0.3, max_missed=5)
total_count_m = 0
start_time = time.time()
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret: break
    frame_idx += 1

    # prepare input
    img_t = transform(frame).to(device)
    with torch.no_grad():
        outputs = mob_model([img_t])[0]  # dict with boxes, scores, labels

    dets = []
    boxes = outputs['boxes'].cpu().numpy()
    scores = outputs['scores'].cpu().numpy()
    labels = outputs['labels'].cpu().numpy()
    for b, s, lab in zip(boxes, scores, labels):
        # COCO label for person is 1
        if lab == 1 and s > 0.4:
            x1,y1,x2,y2 = b.astype(int)
            dets.append([x1,y1,x2,y2])

    tracks = tracker.update(dets)

    # draw
    x1f,y1f,x2f,y2f = FENCE
    cv2.rectangle(frame, (x1f,y1f), (x2f,y2f), (0,255,0), 2)
    for tid, t in tracks.items():
        bx1,by1,bx2,by2 = map(int, t['bbox'])
        cx = (bx1+bx2)//2; cy = (by1+by2)//2
        cv2.rectangle(frame, (bx1,by1),(bx2,by2),(255,0,0),2)
        cv2.putText(frame, f"ID:{tid}", (bx1, max(0,by1-8)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,0),1)
        if (x1f <= cx <= x2f and y1f <= cy <= y2f) and (not t['counted']):
            total_count_m += 1
            t['counted'] = True
            tracker.tracks[tid]['counted'] = True

    cv2.putText(frame, f"MobileNet Count: {total_count_m}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)
    out.write(frame)

cap.release(); out.release()
elapsed_mob = time.time() - start_time
print(f"✅ MobileNet-SSD done — Count: {total_count_m}, Time: {elapsed_mob:.2f}s, Output: {OUTPUT_VIDEO_MOB}")
mobilenet_results = {'count': total_count_m, 'time': elapsed_mob, 'output': OUTPUT_VIDEO_MOB}


از مدل ssdlite320_mobilenet_v3_large موجود در torchvision استفاده می‌کنیم.

سبک و سریع است؛ مناسب برای مقایسه با YOLOv8n.

آستانه نمره (score) را 0.4 قرار دادم؛ در صورت نیاز می‌توان آن را بالاتر برد تا false-positive کمتر شود.

In [ ]:
###Cell D — اجرای OMNI VLM
# ===========================
# Cell D: VLM-based detection (optional) — safe fallback if not available
# ===========================
import importlib, sys
vlm_available = False
time_vlm = None
vlm_count = 0
OUTPUT_VIDEO_VLM = "/content/output_vlm.mp4"

try:
    # Try to import groundingdino from huggingface transformers pipeline or custom libs
    from transformers import AutoProcessor, AutoModelForObjectDetection
    # <-- NOTE: model name must be replaced with a valid HF model that supports object detection grounding.
    # Example placeholder name (may need to be changed to an available checkpoint):
    MODEL_NAME = "ShilongLiu/groundingdino-swinb-coco"  # placeholder; replace if you have exact model
    print("Attempting to load VLM model:", MODEL_NAME)
    processor = AutoProcessor.from_pretrained(MODEL_NAME)
    vlm_model = AutoModelForObjectDetection.from_pretrained(MODEL_NAME)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    vlm_model.to(device)
    vlm_model.eval()
    vlm_available = True
except Exception as e:
    print("⚠️ VLM model not available in this environment or AutoModel config incompatible.")
    print("To enable VLM, install the appropriate HF model and update MODEL_NAME variable to a compatible grounding-detection model.")
    print("Exception:", e)
    vlm_available = False

if vlm_available:
    cap = cv2.VideoCapture(INPUT_VIDEO)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS) or 25.0
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO_VLM, fourcc, fps, (width, height))

    tracker = SimpleTracker(iou_threshold=0.3, max_missed=5)
    vlm_count = 0
    start_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret: break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Prepare processor inputs (text prompt)
        inputs = processor(images=image_rgb, text=["person"], return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = vlm_model(**inputs)
        # Parse detections — parsing code depends on model output structure
        # Try generic fields; if not present, adapt according to the HF model docs
        try:
            boxes = outputs.pred_boxes[0].cpu().numpy()  # may differ per model
            scores = outputs.pred_logits[0].softmax(-1).max(-1).values.cpu().numpy()
        except:
            # fallback: try outputs["boxes"], outputs["scores"]
            boxes = outputs.get("boxes", np.array([]))
            scores = outputs.get("scores", np.array([]))

        dets = []
        for b, s in zip(boxes, scores):
            if s < 0.3: continue
            x1,y1,x2,y2 = b.astype(int)
            dets.append([x1,y1,x2,y2])

        tracks = tracker.update(dets)
        x1f,y1f,x2f,y2f = FENCE
        cv2.rectangle(frame, (x1f,y1f),(x2f,y2f), (0,255,0), 2)
        for tid, t in tracks.items():
            bx1,by1,bx2,by2 = map(int, t['bbox'])
            cx = (bx1+bx2)//2; cy = (by1+by2)//2
            cv2.rectangle(frame, (bx1,by1),(bx2,by2),(0,255,255),2)
            if (x1f <= cx <= x2f and y1f <= cy <= y2f) and (not t['counted']):
                vlm_count += 1
                t['counted'] = True
                tracker.tracks[tid]['counted'] = True

        cv2.putText(frame, f"VLM Count: {vlm_count}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)
        out.write(frame)

    cap.release(); out.release()
    time_vlm = time.time() - start_time
    print(f"✅ VLM done — Count: {vlm_count}, Time: {time_vlm:.2f}s, Output: {OUTPUT_VIDEO_VLM}")
    vlm_results = {'count': vlm_count, 'time': time_vlm, 'output': OUTPUT_VIDEO_VLM}
else:
    print("VLM step skipped. Provide a compatible HF grounding detection model and set MODEL_NAME to run this step.")
    vlm_results = {'count': None, 'time': None, 'output': None}


VLMها روی HuggingFace تنوع زیادی دارند و خروجی هر مدل ساختار مخصوصی دارد. من یک بلوک امن گذاشتم که تلاش می‌کند مدل را بارگذاری کند و اگر incompatible باشد، مرحله را رد می‌کند و راهنمایی می‌دهد که چه کاری باید انجام دهی (تغییر MODEL_NAME به یک مدل مناسب یا نصب پکیج‌های لازم).

اگر مدل VLM در محیط نصب باشد، این بلوک آن را اجرا می‌کند؛ در غیر این صورت skip می‌شود و پروژه ادامه پیدا می‌کند.

این سلول سعی می‌کند یک VLM از HuggingFace مثل GroundingDINO یا مدل detection text-conditioned را بارگذاری کند؛ اگر مدل/پکیج در محیط موجود نباشد، عملیات را هوشمندانه رد می‌کند و به تو پیام راهنما می‌دهد.

In [ ]:
##Cell E — جمع‌آوری نتایج و بنچمارک (جدول + نمودار)

In [ ]:
# ===========================
# Cell E: Benchmark table + plots
# ===========================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

rows = []
# YOLO results
rows.append({
    'Method': 'YOLOv8n',
    'Counted': yolo_results['count'],
    'Time_s': yolo_results['time'],
    'Output': yolo_results['output']
})
# MobileNet
rows.append({
    'Method': 'MobileNet-SSD',
    'Counted': mobilenet_results['count'],
    'Time_s': mobilenet_results['time'],
    'Output': mobilenet_results['output']
})
# VLM (if any)
rows.append({
    'Method': 'OMNI_VLM',
    'Counted': vlm_results.get('count'),
    'Time_s': vlm_results.get('time'),
    'Output': vlm_results.get('output')
})

df = pd.DataFrame(rows)
display(df)

# Plot counts
plt.figure(figsize=(8,4))
sns.barplot(data=df.dropna(subset=['Counted']), x='Method', y='Counted', palette='pastel')
plt.title("Detected / Counted people per method")
plt.ylabel("Count")
plt.show()

# Plot time
plt.figure(figsize=(8,4))
sns.barplot(data=df.dropna(subset=['Time_s']), x='Method', y='Time_s', palette='muted')
plt.title("Processing time (seconds) per method")
plt.ylabel("Seconds")
plt.show()

# Save benchmark CSV
bench_path = "/content/benchmark_results.csv"
df.to_csv(bench_path, index=False)
print("✅ Benchmark saved to", bench_path)


In [ ]:
###Cell F — نکات بهینه‌سازی نهایی و آماده‌سازی README / خروجی‌ها
# ===========================
# Cell F: Final notes, save outputs and README snippet
# ===========================
# فایل‌های خروجی تولید شده:
print("YOLO output:", yolo_results.get('output'))
print("MobileNet output:", mobilenet_results.get('output'))
print("VLM output:", vlm_results.get('output'))

# ذخیره نتایج خلاصه در یک JSON برای مستندسازی
import json
summary = {
    'yolo': yolo_results,
    'mobilenet': mobilenet_results,
    'vlm': vlm_results,
    'fence_coords': FENCE,
    'input_video': INPUT_VIDEO
}
with open('/content/experiment_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Summary saved: /content/experiment_summary.json")

# README snippet (کوتاه) برای گیت‌هاب
readme_snippet = f"""
# Virtual Fence - Experiment Summary

Input video: {INPUT_VIDEO}
Fence coords: {FENCE}

Methods & outputs:
- YOLOv8n: output saved at {yolo_results.get('output')}, time={yolo_results.get('time'):.2f}s, count={yolo_results.get('count')}
- MobileNet-SSD: output saved at {mobilenet_results.get('output')}, time={mobilenet_results.get('time'):.2f}s, count={mobilenet_results.get('count')}
- OMNI VLM: output saved at {vlm_results.get('output')}, time={vlm_results.get('time')}, count={vlm_results.get('count')}

Benchmark CSV: /content/benchmark_results.csv
"""
with open('/content/README_EXPERIMENT.txt', 'w') as f:
    f.write(readme_snippet)
print("✅ README_EXPERIMENT snippet created at /content/README_EXPERIMENT.txt")
